In [23]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from tqdm import tqdm
import os
import PIL.Image as Image

import torchvision
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts

In [24]:
from data import initialize_data, training_transforms, test_transforms # data.py in the same folder
initialize_data('images/') # extracts the zip files, makes a validation set

In [25]:
# All the possible transforms.

# Keep the same
t0 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Scale brightness between the range (1.5,3.5)
t1 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ColorJitter(brightness=2.5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Scale saturation between (1,2)
t2 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ColorJitter(saturation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Scale contrast between (1,1.5)
t3 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ColorJitter(contrast=1.5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Scale hue
t4 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ColorJitter(hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Random horizontal flips
t5 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Random shearing
t6 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomAffine(degrees=20, shear=3),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Random Translation
t7 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomAffine(degrees=10, translate=(0.2,0.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Random perspective change
t8 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Random rotation
t9 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Upscale the image to 75x75 then make a random crop of 50x50
t10 = transforms.Compose([
    transforms.Resize((75, 75)),
    transforms.RandomResizedCrop(size=50),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

t11 = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.TenCrop(50),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))(crop) for crop in crops])),
])

In [5]:
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([
        datasets.ImageFolder('images/train_images', transform=t0),
        datasets.ImageFolder('images/train_images', transform=t1),
        datasets.ImageFolder('images/train_images', transform=t2),
        datasets.ImageFolder('images/train_images', transform=t3),
        datasets.ImageFolder('images/train_images', transform=t4),
        datasets.ImageFolder('images/train_images', transform=t5),
        datasets.ImageFolder('images/train_images', transform=t6),
        datasets.ImageFolder('images/train_images', transform=t7),
        datasets.ImageFolder('images/train_images', transform=t8),
        datasets.ImageFolder('images/train_images', transform=t9),
        datasets.ImageFolder('images/train_images', transform=t10),
    ]),
    batch_size=1024, shuffle=True, num_workers = 8)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('images/val_images', transform=t0),
    batch_size=1024, shuffle=False, num_workers = 8)

In [26]:
device = torch.device('cuda')
nclasses = 43 # GTSRB as 43 classes

In [8]:
class Conv4Net(nn.Module):
    def __init__(self):
        super(Conv4Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*8*8, 256)
        self.fc2 = nn.Linear(256, nclasses)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, training=self.training, p=0.25)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, training=self.training, p=0.25)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.5)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [27]:
conv42 = Conv4Net().to(device)
conv42

Conv4Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4096, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=43, bias=True)
)

In [9]:
def train(epoch, model, optimizer, sched):
    model.train()
    iters = len(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        sched.step(epoch + batch_idx/iters)
        model.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [10]:
state_dict = torch.load('conv42_5000_1500.pth')
conv42.load_state_dict(state_dict)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv42.parameters())
sched = CosineAnnealingWarmRestarts(optimizer, T_0=1)
epochs = 100
exp_name = 'conv42_aug_100_'

In [11]:
for epoch in range(1, epochs + 1):
    train(epoch,conv42,optimizer,sched)
    validation(conv42)
    if epoch%50 == 0:
        model_file = 'Models/' + exp_name + str(epoch) + '.pth'
        torch.save(conv42.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

Train Epoch: 1 [0/353390 (0%)]	Loss: 6.449107
Train Epoch: 1 [10240/353390 (3%)]	Loss: 4.408457
Train Epoch: 1 [20480/353390 (6%)]	Loss: 3.167373
Train Epoch: 1 [30720/353390 (9%)]	Loss: 2.072049
Train Epoch: 1 [40960/353390 (12%)]	Loss: 1.718888
Train Epoch: 1 [51200/353390 (14%)]	Loss: 1.727370
Train Epoch: 1 [61440/353390 (17%)]	Loss: 1.580399
Train Epoch: 1 [71680/353390 (20%)]	Loss: 1.349610
Train Epoch: 1 [81920/353390 (23%)]	Loss: 1.107317
Train Epoch: 1 [92160/353390 (26%)]	Loss: 0.939244
Train Epoch: 1 [102400/353390 (29%)]	Loss: 1.110122
Train Epoch: 1 [112640/353390 (32%)]	Loss: 1.220660
Train Epoch: 1 [122880/353390 (35%)]	Loss: 1.163993
Train Epoch: 1 [133120/353390 (38%)]	Loss: 0.896368
Train Epoch: 1 [143360/353390 (40%)]	Loss: 0.976258
Train Epoch: 1 [153600/353390 (43%)]	Loss: 0.959014
Train Epoch: 1 [163840/353390 (46%)]	Loss: 1.002272
Train Epoch: 1 [174080/353390 (49%)]	Loss: 0.870108
Train Epoch: 1 [184320/353390 (52%)]	Loss: 0.924807
Train Epoch: 1 [194560/353390 

Train Epoch: 5 [153600/353390 (43%)]	Loss: 0.465255
Train Epoch: 5 [163840/353390 (46%)]	Loss: 0.505031
Train Epoch: 5 [174080/353390 (49%)]	Loss: 0.513980
Train Epoch: 5 [184320/353390 (52%)]	Loss: 0.443992
Train Epoch: 5 [194560/353390 (55%)]	Loss: 0.533679
Train Epoch: 5 [204800/353390 (58%)]	Loss: 0.378803
Train Epoch: 5 [215040/353390 (61%)]	Loss: 0.461698
Train Epoch: 5 [225280/353390 (64%)]	Loss: 0.487637
Train Epoch: 5 [235520/353390 (66%)]	Loss: 0.362349
Train Epoch: 5 [245760/353390 (69%)]	Loss: 0.354485
Train Epoch: 5 [256000/353390 (72%)]	Loss: 0.494513
Train Epoch: 5 [266240/353390 (75%)]	Loss: 0.375404
Train Epoch: 5 [276480/353390 (78%)]	Loss: 0.477500
Train Epoch: 5 [286720/353390 (81%)]	Loss: 0.383988
Train Epoch: 5 [296960/353390 (84%)]	Loss: 0.431769
Train Epoch: 5 [307200/353390 (87%)]	Loss: 0.488616
Train Epoch: 5 [317440/353390 (90%)]	Loss: 0.378352
Train Epoch: 5 [327680/353390 (92%)]	Loss: 0.455393
Train Epoch: 5 [337920/353390 (95%)]	Loss: 0.476367
Train Epoch:

Train Epoch: 9 [296960/353390 (84%)]	Loss: 0.288193
Train Epoch: 9 [307200/353390 (87%)]	Loss: 0.355619
Train Epoch: 9 [317440/353390 (90%)]	Loss: 0.294100
Train Epoch: 9 [327680/353390 (92%)]	Loss: 0.352786
Train Epoch: 9 [337920/353390 (95%)]	Loss: 0.357241
Train Epoch: 9 [348160/353390 (98%)]	Loss: 0.274956

Validation set: Average loss: 0.0003, Accuracy: 3555/3870 (92%)

Train Epoch: 10 [0/353390 (0%)]	Loss: 0.321949
Train Epoch: 10 [10240/353390 (3%)]	Loss: 0.317011
Train Epoch: 10 [20480/353390 (6%)]	Loss: 0.375047
Train Epoch: 10 [30720/353390 (9%)]	Loss: 0.311938
Train Epoch: 10 [40960/353390 (12%)]	Loss: 0.325325
Train Epoch: 10 [51200/353390 (14%)]	Loss: 0.364843
Train Epoch: 10 [61440/353390 (17%)]	Loss: 0.348240
Train Epoch: 10 [71680/353390 (20%)]	Loss: 0.306763
Train Epoch: 10 [81920/353390 (23%)]	Loss: 0.340954
Train Epoch: 10 [92160/353390 (26%)]	Loss: 0.327188
Train Epoch: 10 [102400/353390 (29%)]	Loss: 0.367405
Train Epoch: 10 [112640/353390 (32%)]	Loss: 0.304940
Trai

Train Epoch: 14 [51200/353390 (14%)]	Loss: 0.326721
Train Epoch: 14 [61440/353390 (17%)]	Loss: 0.316057
Train Epoch: 14 [71680/353390 (20%)]	Loss: 0.219560
Train Epoch: 14 [81920/353390 (23%)]	Loss: 0.261269
Train Epoch: 14 [92160/353390 (26%)]	Loss: 0.241654
Train Epoch: 14 [102400/353390 (29%)]	Loss: 0.270652
Train Epoch: 14 [112640/353390 (32%)]	Loss: 0.246911
Train Epoch: 14 [122880/353390 (35%)]	Loss: 0.328259
Train Epoch: 14 [133120/353390 (38%)]	Loss: 0.261210
Train Epoch: 14 [143360/353390 (40%)]	Loss: 0.348490
Train Epoch: 14 [153600/353390 (43%)]	Loss: 0.249408
Train Epoch: 14 [163840/353390 (46%)]	Loss: 0.236851
Train Epoch: 14 [174080/353390 (49%)]	Loss: 0.257653
Train Epoch: 14 [184320/353390 (52%)]	Loss: 0.210061
Train Epoch: 14 [194560/353390 (55%)]	Loss: 0.307896
Train Epoch: 14 [204800/353390 (58%)]	Loss: 0.298935
Train Epoch: 14 [215040/353390 (61%)]	Loss: 0.253437
Train Epoch: 14 [225280/353390 (64%)]	Loss: 0.207475
Train Epoch: 14 [235520/353390 (66%)]	Loss: 0.20546

Train Epoch: 18 [174080/353390 (49%)]	Loss: 0.275778
Train Epoch: 18 [184320/353390 (52%)]	Loss: 0.281916
Train Epoch: 18 [194560/353390 (55%)]	Loss: 0.240110
Train Epoch: 18 [204800/353390 (58%)]	Loss: 0.216264
Train Epoch: 18 [215040/353390 (61%)]	Loss: 0.205088
Train Epoch: 18 [225280/353390 (64%)]	Loss: 0.259314
Train Epoch: 18 [235520/353390 (66%)]	Loss: 0.254078
Train Epoch: 18 [245760/353390 (69%)]	Loss: 0.228881
Train Epoch: 18 [256000/353390 (72%)]	Loss: 0.191742
Train Epoch: 18 [266240/353390 (75%)]	Loss: 0.222945
Train Epoch: 18 [276480/353390 (78%)]	Loss: 0.269612
Train Epoch: 18 [286720/353390 (81%)]	Loss: 0.252883
Train Epoch: 18 [296960/353390 (84%)]	Loss: 0.268264
Train Epoch: 18 [307200/353390 (87%)]	Loss: 0.269861
Train Epoch: 18 [317440/353390 (90%)]	Loss: 0.250839
Train Epoch: 18 [327680/353390 (92%)]	Loss: 0.226491
Train Epoch: 18 [337920/353390 (95%)]	Loss: 0.222581
Train Epoch: 18 [348160/353390 (98%)]	Loss: 0.195976

Validation set: Average loss: 0.0003, Accurac

...

Train Epoch: 83 [40960/353390 (12%)]	Loss: 0.102049
Train Epoch: 83 [51200/353390 (14%)]	Loss: 0.112735
Train Epoch: 83 [61440/353390 (17%)]	Loss: 0.124429
Train Epoch: 83 [71680/353390 (20%)]	Loss: 0.094922
Train Epoch: 83 [81920/353390 (23%)]	Loss: 0.127977
Train Epoch: 83 [92160/353390 (26%)]	Loss: 0.112100
Train Epoch: 83 [102400/353390 (29%)]	Loss: 0.107151
Train Epoch: 83 [112640/353390 (32%)]	Loss: 0.134017
Train Epoch: 83 [122880/353390 (35%)]	Loss: 0.131067
Train Epoch: 83 [133120/353390 (38%)]	Loss: 0.144122
Train Epoch: 83 [143360/353390 (40%)]	Loss: 0.114677
Train Epoch: 83 [153600/353390 (43%)]	Loss: 0.109380
Train Epoch: 83 [163840/353390 (46%)]	Loss: 0.103248
Train Epoch: 83 [174080/353390 (49%)]	Loss: 0.123309
Train Epoch: 83 [184320/353390 (52%)]	Loss: 0.082418
Train Epoch: 83 [194560/353390 (55%)]	Loss: 0.117216
Train Epoch: 83 [204800/353390 (58%)]	Loss: 0.099441
Train Epoch: 83 [215040/353390 (61%)]	Loss: 0.091581
Train Epoch: 83 [225280/353390 (64%)]	Loss: 0.114262

Train Epoch: 87 [163840/353390 (46%)]	Loss: 0.124752
Train Epoch: 87 [174080/353390 (49%)]	Loss: 0.118495
Train Epoch: 87 [184320/353390 (52%)]	Loss: 0.113658
Train Epoch: 87 [194560/353390 (55%)]	Loss: 0.116648
Train Epoch: 87 [204800/353390 (58%)]	Loss: 0.133224
Train Epoch: 87 [215040/353390 (61%)]	Loss: 0.133188
Train Epoch: 87 [225280/353390 (64%)]	Loss: 0.122131
Train Epoch: 87 [235520/353390 (66%)]	Loss: 0.074711
Train Epoch: 87 [245760/353390 (69%)]	Loss: 0.093716
Train Epoch: 87 [256000/353390 (72%)]	Loss: 0.118234
Train Epoch: 87 [266240/353390 (75%)]	Loss: 0.084495
Train Epoch: 87 [276480/353390 (78%)]	Loss: 0.112331
Train Epoch: 87 [286720/353390 (81%)]	Loss: 0.117960
Train Epoch: 87 [296960/353390 (84%)]	Loss: 0.102168
Train Epoch: 87 [307200/353390 (87%)]	Loss: 0.088499
Train Epoch: 87 [317440/353390 (90%)]	Loss: 0.085599
Train Epoch: 87 [327680/353390 (92%)]	Loss: 0.105001
Train Epoch: 87 [337920/353390 (95%)]	Loss: 0.089735
Train Epoch: 87 [348160/353390 (98%)]	Loss: 0.

Train Epoch: 91 [276480/353390 (78%)]	Loss: 0.104515
Train Epoch: 91 [286720/353390 (81%)]	Loss: 0.142372
Train Epoch: 91 [296960/353390 (84%)]	Loss: 0.129197
Train Epoch: 91 [307200/353390 (87%)]	Loss: 0.110640
Train Epoch: 91 [317440/353390 (90%)]	Loss: 0.114423
Train Epoch: 91 [327680/353390 (92%)]	Loss: 0.105195
Train Epoch: 91 [337920/353390 (95%)]	Loss: 0.084602
Train Epoch: 91 [348160/353390 (98%)]	Loss: 0.107311

Validation set: Average loss: 0.0002, Accuracy: 3656/3870 (94%)

Train Epoch: 92 [0/353390 (0%)]	Loss: 0.115346
Train Epoch: 92 [10240/353390 (3%)]	Loss: 0.082964
Train Epoch: 92 [20480/353390 (6%)]	Loss: 0.143805
Train Epoch: 92 [30720/353390 (9%)]	Loss: 0.118176
Train Epoch: 92 [40960/353390 (12%)]	Loss: 0.159747
Train Epoch: 92 [51200/353390 (14%)]	Loss: 0.128004
Train Epoch: 92 [61440/353390 (17%)]	Loss: 0.122027
Train Epoch: 92 [71680/353390 (20%)]	Loss: 0.099558
Train Epoch: 92 [81920/353390 (23%)]	Loss: 0.157055
Train Epoch: 92 [92160/353390 (26%)]	Loss: 0.10460

Train Epoch: 96 [20480/353390 (6%)]	Loss: 0.100944
Train Epoch: 96 [30720/353390 (9%)]	Loss: 0.138826
Train Epoch: 96 [40960/353390 (12%)]	Loss: 0.107044
Train Epoch: 96 [51200/353390 (14%)]	Loss: 0.125996
Train Epoch: 96 [61440/353390 (17%)]	Loss: 0.133767
Train Epoch: 96 [71680/353390 (20%)]	Loss: 0.101315
Train Epoch: 96 [81920/353390 (23%)]	Loss: 0.106369
Train Epoch: 96 [92160/353390 (26%)]	Loss: 0.073495
Train Epoch: 96 [102400/353390 (29%)]	Loss: 0.112448
Train Epoch: 96 [112640/353390 (32%)]	Loss: 0.089488
Train Epoch: 96 [122880/353390 (35%)]	Loss: 0.112015
Train Epoch: 96 [133120/353390 (38%)]	Loss: 0.084972
Train Epoch: 96 [143360/353390 (40%)]	Loss: 0.118732
Train Epoch: 96 [153600/353390 (43%)]	Loss: 0.103486
Train Epoch: 96 [163840/353390 (46%)]	Loss: 0.098098
Train Epoch: 96 [174080/353390 (49%)]	Loss: 0.086742
Train Epoch: 96 [184320/353390 (52%)]	Loss: 0.116199
Train Epoch: 96 [194560/353390 (55%)]	Loss: 0.107688
Train Epoch: 96 [204800/353390 (58%)]	Loss: 0.093342
Tra

Train Epoch: 100 [133120/353390 (38%)]	Loss: 0.156432
Train Epoch: 100 [143360/353390 (40%)]	Loss: 0.128498
Train Epoch: 100 [153600/353390 (43%)]	Loss: 0.113289
Train Epoch: 100 [163840/353390 (46%)]	Loss: 0.111442
Train Epoch: 100 [174080/353390 (49%)]	Loss: 0.105961
Train Epoch: 100 [184320/353390 (52%)]	Loss: 0.145814
Train Epoch: 100 [194560/353390 (55%)]	Loss: 0.110163
Train Epoch: 100 [204800/353390 (58%)]	Loss: 0.084754
Train Epoch: 100 [215040/353390 (61%)]	Loss: 0.117779
Train Epoch: 100 [225280/353390 (64%)]	Loss: 0.094028
Train Epoch: 100 [235520/353390 (66%)]	Loss: 0.118651
Train Epoch: 100 [245760/353390 (69%)]	Loss: 0.096584
Train Epoch: 100 [256000/353390 (72%)]	Loss: 0.057211
Train Epoch: 100 [266240/353390 (75%)]	Loss: 0.111905
Train Epoch: 100 [276480/353390 (78%)]	Loss: 0.110927
Train Epoch: 100 [286720/353390 (81%)]	Loss: 0.115508
Train Epoch: 100 [296960/353390 (84%)]	Loss: 0.103597
Train Epoch: 100 [307200/353390 (87%)]	Loss: 0.084007
Train Epoch: 100 [317440/353

# Resnet18

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=False)
nclasses = 43
num_ftrs = resnet18.fc.in_features
resnet18.fc = torch.nn.Linear(num_ftrs, nclasses)
resnet18.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=num_ftrs,
        out_features=nclasses
    ),
    nn.LogSoftmax()
)

resnet18 = resnet18.to(device)

In [ ]:
def train(epoch, model, optimizer,sched):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        model.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        sched.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters())
# exp_lr_scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
epochs = 100
exp_name = 'resnet18_100_'

In [ ]:
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.0007)

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch,resnet18,optimizer)
    validation(resnet18)
    if epoch%5 == 0:
        model_file = 'Models/' + exp_name + str(epoch) + '.pth'
        torch.save(resnet18.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

In [ ]:
epochs = 100
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.0001)
for epoch in range(1, epochs + 1):
    train(epoch,resnet18,optimizer)
    validation(resnet18)
    if epoch%5 == 0:
        model_file = 'Models/' + exp_name + str(epoch) + '.pth'
        torch.save(resnet18.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

# ResNext 50

In [ ]:
resnext = torchvision.models.resnext50_32x4d()
nclasses = 43
num_ftrs = resnext.fc.in_features
resnext.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=num_ftrs,
        out_features=nclasses
    ),
    nn.LogSoftmax()
)

resnext = resnext.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnext.parameters())
epochs = 100
exp_name = 'resnext_100_'

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch,resnext,optimizer,sched)
    validation(resnext)
    if epoch%5 == 0:
        model_file = 'Models/' + exp_name + str(epoch) + '.pth'
        torch.save(resnext.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

In [ ]:
state_dict = torch.load('resnext2_500_170.pth')
resnext.load_state_dict(state_dict)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnext.fc.parameters(), lr=0.001, momentum=0.9)
sched = StepLR(optimizer, step_size=20, gamma=0.1)
epochs = 500
exp_name = 'resnext3_500_'

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch,resnext,optimizer,sched)
    validation(resnext)
    if epoch%5 == 0:
        model_file = 'Models/' + exp_name + str(epoch) + '.pth'
        torch.save(resnext.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

## Resnext - 1000 iterations

In [ ]:
state_dict = torch.load('resnext3_500_500.pth')
resnext.load_state_dict(state_dict)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnext.fc.parameters(), lr=0.001, momentum=0.9)
sched = StepLR(optimizer, step_size=75, gamma=0.1)
epochs = 1000
exp_name = 'resnext4_1000_'

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch,resnext,optimizer,sched)
    validation(resnext)
    if epoch%50 == 0:
        print(optimizer.param_groups[0]['lr'])
        model_file = exp_name + str(epoch) + '.pth'
        torch.save(resnext.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

# Eval

In [13]:
state_dict = torch.load('conv42_aug_100_100.pth')
model = conv42
model.load_state_dict(state_dict)
model.eval()
exp_name = 'conv42_tta_aug_100_100'

In [15]:
test_dir = 'images/test_images'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

outfile = exp_name+'_out.csv'

output_file = open(outfile, "w")
output_file.write("Filename,ClassId\n")
for f in tqdm(os.listdir(test_dir)):
    if 'ppm' in f:
        data1 = t10(pil_loader(test_dir + '/' + f))
        data2 = t1((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data3 = t2((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data4 = t3((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data5 = t4((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data6 = t5((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data7 = t6((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data8 = t7((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data9 = t8((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data10 = t9((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        final_input = torch.cat((data1,data2,data3,data4,data5,data6,data7,data8,data9,data10),0)
         
        ntrans, c, h, w = final_input.size()
        temp_output = model(Variable(final_input.view(-1, c, h, w)).to(device))
        output = temp_output.mean(0)
        pred = output.data.max(0, keepdim=True)[1]
    
        file_id = f[0:5]
        output_file.write("%s,%d\n" % (file_id, pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle '
      'competition at https://www.kaggle.com/c/nyu-cv-fall-2018/')
        

100%|██████████| 12630/12630 [02:49<00:00, 74.39it/s]

Succesfully wrote conv42_tta_aug_100_100_out.csv, you can upload this file to the kaggle competition at https://www.kaggle.com/c/nyu-cv-fall-2018/


### Ensembling

In [34]:
state_dict_1 = torch.load('Models/conv42_aug_100_100.pth')
model1 = conv42
model1.load_state_dict(state_dict_1)
model1.eval()

state_dict_2 = torch.load('Models/mcnn42_aug_500_100.pth')
model2 = mcnn
model2.load_state_dict(state_dict_2)
model2.eval()

exp_name = 'ensemble_5'

In [35]:
test_dir = 'images/test_images'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

outfile = 'Output/'+exp_name+'_out.csv'

output_file = open(outfile, "w")
output_file.write("Filename,ClassId\n")
for f in tqdm(os.listdir(test_dir)):
    if 'ppm' in f:
        data1 = t11(pil_loader(test_dir + '/' + f))
        data2 = t1((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data3 = t2((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data4 = t3((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data5 = t4((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data6 = t5((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data7 = t6((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data8 = t7((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data9 = t8((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data10 = t9((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data11 = t10((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data12 = t0((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        
        final_input = torch.cat((data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12),0)
         
        ntrans, c, h, w = final_input.size()
        temp_output_1 = model1(Variable(final_input.view(-1, c, h, w)).to(device))
        temp_output_2 = model2(Variable(final_input.view(-1, c, h, w)).to(device))
        
        ensemble_res = torch.cat((temp_output_1, temp_output_2),0)
        output = ensemble_res.mean(0)
        pred = output.data.max(0, keepdim=True)[1]
    
        file_id = f[0:5]
        output_file.write("%s,%d\n" % (file_id, pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle '
      'competition at https://www.kaggle.com/c/nyu-cv-fall-2018/')
        

100%|██████████| 12630/12630 [03:17<00:00, 64.10it/s]

Succesfully wrote Output/ensemble_5_out.csv, you can upload this file to the kaggle competition at https://www.kaggle.com/c/nyu-cv-fall-2018/
